In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector as mdb
import ujson as js
import torch as pt
import tqdm
import mysql.connector as mdb

sns.set()

from utils.market_parser import market_smooth
from utils.sql_pullers import get_types

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
def mat_eff_calc(mat_quant, me_level):
    quant_actual = np.maximum(1, np.ceil(np.round(mat_quant * ((100 - me_level) / 100), 2))).astype(int)
    return quant_actual

In [3]:
with open('./utils/sql_calls.json') as file:
    sql = js.load(file)

mat_region_id = 10000043
sale_region_id = 10000052
#sale_region_id = 10000043

h = 14
markets = {}
prices = {}

market_start = '2019-01-01'

In [4]:
with open('./../settings/maria_login.json') as file:
    conn = mdb.connect(**js.load(file))

In [5]:
corp_bps = pd.read_sql(sql['corp']['bpos'], conn, index_col='bp_item_id')
corp_bps = corp_bps.loc[corp_bps[['bp_type_id', 'mat_eff', 'time_eff']].drop_duplicates().index]
corp_bps.index = corp_bps.index.astype(str)

In [6]:
corp_orders = pd.read_sql(sql['corp']['orders'], conn, index_col='order_id')

In [7]:
bp_data = {}
for key, script in sql['bp'].items():
    bp_data[key] = pd.read_sql(script.format(type_ids=','.join(corp_bps['bp_type_id'].astype(str))), conn)

In [8]:
invent = {
    'mats': {
        'quant': corp_bps.reset_index().merge(
            bp_data['mats'].loc[bp_data['mats']['act_id'] == 8],
            on='bp_type_id'
        )[['bp_item_id', 'bp_type_id', 'mat_eff', 'mat_type_id', 'mat_quant']].pivot(
            index='bp_item_id',
            columns='mat_type_id',
            values='mat_quant'
        ).fillna(0)
    }
}

In [9]:
markets['invent'] = market_smooth(mat_region_id, invent['mats']['quant'].columns, h, start_date=market_start)

prices[mat_region_id] = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['invent'][mat_region_id].items()]
).set_index('type_id')

invent['mats']['price'] = pd.concat([
    (prices[mat_region_id]['avg_price'] * invent['mats']['quant']).sum(axis=1),
    (prices[mat_region_id]['variance'] * (invent['mats']['quant']**2)).sum(axis=1)
], axis=1)
invent['mats']['price'].columns = prices[mat_region_id].columns

/home/calvinxc1/conda/envs/analysis/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
invent['prods'] = corp_bps.reset_index().merge(
    bp_data['prods'].loc[bp_data['prods']['act_id'] == 8],
    on='bp_type_id'
)[['bp_item_id', 'bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']]
invent['prods']['avg_price_run'] = invent['prods']['bp_item_id'].map(invent['mats']['price']['avg_price'])
invent['prods']['variance_run'] = invent['prods']['bp_item_id'].map(invent['mats']['price']['variance'])
invent['prods']['avg_price_unit'] = invent['prods']['avg_price_run'] / invent['prods'][['prod_quant', 'prod_prob']].product(axis=1)
invent['prods']['variance_unit'] = invent['prods']['variance_run'] * (invent['prods'][['prod_quant', 'prod_prob']].product(axis=1)**2)
invent['prods'].index = invent['prods']['bp_item_id'] + ':' + invent['prods']['prod_type_id'].astype(str)
invent['prods'].index.name = 'bp_item_id'
invent['prods'].drop(['bp_item_id', 'bp_type_id'], axis=1, inplace = True)

In [11]:
invent_bps = corp_bps.loc[invent['prods'].index.to_series().apply(lambda x: x.split(':')[0])]
invent_bps.index = invent['prods'].index
invent_bps['bp_type_id'] = invent['prods']['prod_type_id']
invent_bps['mat_eff'] = 2
invent_bps['time_eff'] = 4
corp_bps = corp_bps.append(invent_bps)
corp_bps.index.name = 'bp_item_id'
del invent_bps

In [12]:
for key, sql in sql['bp'].items():
    bp_data[key] = bp_data[key].append(
        pd.read_sql(sql.format(type_ids=','.join(invent['prods']['prod_type_id'].unique().astype(str))), conn),
        ignore_index=True
    ).drop_duplicates().reset_index(drop=True)

In [13]:
build = {
    'count': corp_bps.reset_index().merge(
        bp_data['mats'].loc[bp_data['mats']['act_id'] == 1],
        on='bp_type_id'
    )[['bp_item_id', 'bp_type_id', 'mat_eff', 'mat_type_id', 'mat_quant']]
}
build['count']['mat_actual'] = np.maximum(1, np.ceil(np.round(build['count']['mat_quant'] * ((100 - build['count']['mat_eff']) / 100), 2)))
build['count'] = build['count'].pivot(index='bp_item_id', columns='mat_type_id', values='mat_actual').fillna(0)

In [14]:
sub_prods = bp_data['prods'].loc[
    (bp_data['prods']['prod_type_id'].isin(build['count'].columns)) & (bp_data['prods']['act_id'] == 1)
][['bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']].reset_index(drop=True)
sub_prods = sub_prods.merge(bp_data['mats'].loc[bp_data['mats']['act_id'] == 1, ['bp_type_id', 'mat_type_id', 'mat_quant']], on='bp_type_id')
sub_prods = corp_bps.reset_index()[['bp_item_id', 'bp_type_id', 'mat_eff']].merge(sub_prods, on='bp_type_id')
sub_prods['mat_actual'] = mat_eff_calc(sub_prods['mat_quant'], sub_prods['mat_eff'])
sub_prods['mat_unit'] = sub_prods['mat_actual'] / (sub_prods['prod_quant'] * sub_prods['prod_prob'])
sub_prods['bp_item_id.prod_type_id'] = sub_prods['bp_item_id'] + '.' + sub_prods['prod_type_id'].astype(str)
sub_prods = sub_prods.pivot(index='bp_item_id.prod_type_id', columns='mat_type_id', values='mat_unit').fillna(0)
sub_prods.index = pd.Series([int(col.split('.')[1]) for col in sub_prods.index], name='prod_type_id')

active = True
while active:
    build_mod = build['count'][sub_prods.index] @ sub_prods
    build['count'][sub_prods.index] = 0
    build['count'].loc[build_mod.index, build_mod.columns] += build_mod
    build['count'] = build['count'].loc[:, build['count'].sum(axis=0) > 0]
    
    sub_prods = bp_data['prods'].loc[
        (bp_data['prods']['prod_type_id'].isin(build['count'].columns)) & (bp_data['prods']['act_id'] == 1)
    ][['bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']].reset_index(drop=True)
    sub_prods = sub_prods.merge(bp_data['mats'].loc[bp_data['mats']['act_id'] == 1, ['bp_type_id', 'mat_type_id', 'mat_quant']], on='bp_type_id')
    sub_prods = corp_bps.reset_index()[['bp_item_id', 'bp_type_id', 'mat_eff']].merge(sub_prods, on='bp_type_id')
    sub_prods['mat_actual'] = mat_eff_calc(sub_prods['mat_quant'], sub_prods['mat_eff'])
    sub_prods['mat_unit'] = sub_prods['mat_actual'] / (sub_prods['prod_quant'] * sub_prods['prod_prob'])
    sub_prods['bp_item_id.prod_type_id'] = sub_prods['bp_item_id'] + '.' + sub_prods['prod_type_id'].astype(str)
    sub_prods = sub_prods.pivot(index='bp_item_id.prod_type_id', columns='mat_type_id', values='mat_unit').fillna(0)
    sub_prods.index = pd.Series([int(col.split('.')[1]) for col in sub_prods.index], name='prod_type_id')
    
    if len(sub_prods) == 0: active = False

In [15]:
markets['build'] = market_smooth(mat_region_id, build['count'].columns, h, start_date=market_start)

prices[mat_region_id] = prices[mat_region_id].append(pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['build'][mat_region_id].items()]
).set_index('type_id')).drop_duplicates().sort_index()

In [16]:
build['price'] = pd.concat([
    (build['count'] * prices[mat_region_id]['avg_price']).sum(axis=1),
    ((build['count']**2) * prices[mat_region_id]['variance']).sum(axis=1)
], axis=1)
build['price'].columns = ['%s_run' % col for col in prices[mat_region_id].columns]

In [17]:
build['prods'] = corp_bps.reset_index().merge(
    bp_data['prods'].loc[bp_data['prods']['act_id'] == 1],
    on='bp_type_id'
)[['bp_item_id', 'prod_type_id', 'prod_quant', 'prod_prob']]
build['prods']['avg_price_run'] = build['prods']['bp_item_id'].map(build['price']['avg_price_run'])
build['prods']['variance_run'] = build['prods']['bp_item_id'].map(build['price']['variance_run'])
build['prods'].set_index('bp_item_id', inplace=True)

In [18]:
build['cost'] = build['prods'][['prod_type_id', 'prod_quant', 'prod_prob', 'avg_price_run', 'variance_run']].join(
    invent['prods'][['avg_price_unit', 'variance_unit']]
).fillna(0)
build['cost'].rename(columns={'avg_price_unit':'avg_price_invent', 'variance_unit':'variance_invent'}, inplace=True)

In [19]:
build['cost']['avg_price_unit'] = (
    build['cost']['avg_price_run'] + build['cost']['avg_price_invent']
) / (
    build['cost']['prod_quant'] * build['cost']['prod_prob']
)
build['cost']['variance_unit'] = (
    build['cost']['variance_run'] + build['cost']['variance_invent']
) / (
    (build['cost']['prod_quant'] * build['cost']['prod_prob'])**2
)

In [20]:
markets['sale'] = market_smooth(sale_region_id, build['cost']['prod_type_id'].unique(), h, start_date=market_start)

prices[sale_region_id] = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['sale'][sale_region_id].items()]
).set_index('type_id')

In [21]:
build['cost']['avg_price_sale'] = build['cost']['prod_type_id'].map(prices[sale_region_id]['avg_price'])
build['cost']['variance_sale'] = build['cost']['prod_type_id'].map(prices[sale_region_id]['variance'])
build['cost']['avg_profit'] = build['cost']['avg_price_sale'] - build['cost']['avg_price_unit']
build['cost']['profit_ratio'] = build['cost']['avg_profit'] / build['cost']['avg_price_unit']
build['cost']['variance_profit'] = build['cost']['variance_sale'] + build['cost']['variance_unit']
build['cost']['stdev_profit'] = np.sqrt(build['cost']['variance_profit'])
build['cost']['z_profit'] = build['cost']['avg_profit'] / (np.sqrt(build['cost']['variance_sale']) + np.sqrt(build['cost']['variance_unit']))

In [22]:
types = get_types(build['cost']['prod_type_id'])
build['cost']['prod_type_name'] = build['cost']['prod_type_id'].map(types['type_name'])
build['cost']['prod_group_name'] = build['cost']['prod_type_id'].map(types['group_name'])
build['cost']['prod_cat_name'] = build['cost']['prod_type_id'].map(types['category_name'])

In [23]:
volume = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_volume': vals[h]['level'].iloc[-1]['volume'],
        'variance': vals[h]['var'].iloc[-1]['volume']
    } for type_id, vals in markets['sale'][sale_region_id].items()]
).set_index('type_id')

build['cost']['volume'] = build['cost']['prod_type_id'].map(volume['avg_volume'])
build['cost']['vol_var'] = build['cost']['prod_type_id'].map(volume['variance'])
build['cost']['avg_value'] = build['cost']['avg_profit'] * build['cost']['volume']
build['cost']['var_value'] = (
    build['cost']['variance_profit'] * build['cost']['vol_var']
) + (
    build['cost']['variance_profit'] * (build['cost']['volume']**2)
) + (
    (build['cost']['avg_profit']**2) * build['cost']['vol_var']
)
build['cost']['stdev_value'] = np.sqrt(build['cost']['var_value'])

In [24]:
build['cost']['active_order'] = build['cost']['prod_type_id'].isin(corp_orders['type_id'])

In [25]:
build['cost'].loc[
    ~build['cost']['active_order'],
    [
        'prod_cat_name', 'prod_group_name', 'prod_type_name', 'avg_price_unit',
        'avg_price_sale', 'avg_profit', 'profit_ratio', 'volume', 'avg_value'
    ]
].sort_values('avg_value', ascending=False).iloc[:50]

,prod_cat_name,prod_group_name,prod_type_name,avg_price_unit,avg_price_sale,avg_profit,profit_ratio,volume,avg_value
bp_item_id,,,,,,,,,
278129470,Ship,Battleship,Armageddon,"130,266,721.85","159,755,266.26","29,488,544.41",0.23,0.27,"7,921,063.29"
470571935:12817,Charge,Advanced Pulse Laser Crystal,Conflagration L,"235,742.01","664,119.28","428,377.27",1.82,12.43,"5,325,720.93"
1010832340145:33698,Ship,Expedition Frigate,Prospect,"23,039,814.71","36,624,656.25","13,584,841.54",0.59,0.17,"2,274,621.87"
443581437:30837,Module,Salvager,Salvager II,"950,925.60","1,605,482.40","654,556.80",0.69,2.91,"1,907,258.86"
962734487:12105,Module,Cloaking Device,Covert Ops Cloaking Device II,"5,167,615.98","7,527,828.07","2,360,212.09",0.46,0.37,"865,844.35"
1532766410:21639,Drone,Combat Drone,Vespa II,"725,491.27","983,145.17","257,653.91",0.36,3.36,"864,429.66"
567672053:2176,Drone,Combat Drone,Infiltrator II,"728,617.06","1,281,185.01","552,567.95",0.76,1.44,"796,131.45"
519249209:2421,Module,Missile Launcher Torpedo,Torpedo Launcher II,"4,891,649.83","6,998,635.88","2,106,986.05",0.43,0.36,"761,856.59"
1532765598:21641,Drone,Combat Drone,Valkyrie II,"724,358.21","1,139,004.04","414,645.84",0.57,1.69,"701,367.67"


In [26]:
with open('./../settings/maria_login.json') as file:
    conn = mdb.connect(**js.load(file))

with open('./utils/sql.json') as file:
    undercuts = pd.read_sql(js.load(file)['undercuts'], conn, index_col = 'order_id')
undercuts['market_volume_sum'] = undercuts['market_volume_sum'].astype(int)
types = types.append(get_types([type_id for type_id in undercuts['type_id'].unique() if type_id not in types.index])).sort_index()
undercuts.insert(
    undercuts.columns.get_loc('type_id') + 1,
    'type_name',
    undercuts['type_id'].map(types['type_name'])
)

conn.close()

thresh = 0.1

mask = undercuts['volume'] * thresh <= undercuts['market_volume_sum']

(2, 9)


,type_id,type_name,region_id,price,volume,market_order_count,market_price_min,market_volume_sum,market_volume_max
order_id,,,,,,,,,
5482099217,31454,Small Energy Collision Accelerator I,10000052,"97,500.00",8,1,1.00,1,1
5499147978,31238,Small Memetic Algorithm Bank I,10000052,"125,000.00",10,1,"112,000.00",10,10


In [ ]:
print(undercuts.loc[mask].shape)
undercuts.loc[mask].sort_values('type_name', ascending=True).iloc[:50]

In [27]:
print(undercuts.loc[~mask].shape)
undercuts.loc[~mask].sort_values('type_name', ascending=True).iloc[:50]

(60, 9)


,type_id,type_name,region_id,price,volume,market_order_count,market_price_min,market_volume_sum,market_volume_max
order_id,,,,,,,,,
5483836089,12344,200mm Railgun I,10000052,"95,000.00",100,1,"86,000.00",4,4
5504398645,493,720mm Howitzer Artillery I,10000052,"400,000.00",100,1,"360,000.00",1,1
5483836091,561,75mm Gatling Rail I,10000052,"17,500.00",92,1,"2,000.00",3,3
5482099146,42832,Armor Energizing Charge,10000052,600.00,6900,1,540.00,470,470
5503605315,608,Atron,10000052,"245,000.00",25,1,"200,000.00",1,1
5482099152,3554,Cap Booster 100,10000052,750.00,4652,1,680.00,50,50
5482099153,11285,Cap Booster 200,10000052,"21,750.00",2263,1,"19,000.00",152,152
5484589981,194,Carbonized Lead L,10000052,75.00,100000,1,69.00,4000,4000
5479289549,178,Carbonized Lead S,10000052,18.75,65000,1,17.00,2800,2800
